## Final Notebook|| CASCADE CUP '22


### 1. Data Cleaning

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=0)

In [3]:
train_labels=data_train["cancelled"]
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders',
                            'delivered_time','cancelled','order_id'],axis=1)
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [4]:
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

###  2. TWEAK THE MODEL

In [5]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(3,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  opt = tf.keras.optimizers.SGD(lr=0.0002)
  model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [15]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=4, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Epoch 1/4
  15/1000 [..............................] - ETA: 3s - loss: 1.9564 - accuracy: 0.0514  

2022-02-02 18:11:46.178731: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - ETA: 0s - loss: 1.4373 - accuracy: 0.1790

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-02 18:11:50.251682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - 5s 5ms/step - loss: 1.4373 - accuracy: 0.1790 - val_loss: 1.0840 - val_accuracy: 0.3654
Epoch 2/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9183 - accuracy: 0.5181 - val_loss: 0.7916 - val_accuracy: 0.6404
Epoch 3/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.7225 - accuracy: 0.7156 - val_loss: 0.6636 - val_accuracy: 0.7882
Epoch 4/4
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6222 - accuracy: 0.8336 - val_loss: 0.5866 - val_accuracy: 0.8774


In [16]:
model.evaluate(X_test,y_test)

2813/2813 [==============================] - 9s 3ms/step - loss: 0.5882 - accuracy: 0.8752


[0.5881550908088684, 0.875177800655365]

### 3. CSV SUBMISSION

In [17]:
test=pd.read_csv("data/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','undelivered_orders',
                            'cancelled','order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)

In [18]:
ans=model.predict(test)

2022-02-02 18:12:19.321740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [19]:
sum(ans)/len(ans)

array([0.43140802])

In [20]:
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)


In [21]:
data.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [22]:
data.to_csv("data/model1_10.csv",index=False)